In [1]:
!pip install git+https://github.com/BakyLedos/fast-bert.git

  Cloning https://github.com/BakyLedos/fast-bert.git to /tmp/pip-req-build-f7f1_lw7
  Running command git clone --filter=blob:none --quiet https://github.com/BakyLedos/fast-bert.git /tmp/pip-req-build-f7f1_lw7
  Resolved https://github.com/BakyLedos/fast-bert.git to commit 65984cc8c1a9d1e62db5ed4effd3f6ed01d4da0f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!cp /content/drive/MyDrive/apex.zip .
!unzip apex.zip

--2023-10-22 15:22:53--  http://www.itspaces.tech/apex.zip
Resolving www.itspaces.tech (www.itspaces.tech)... 51.38.177.87
Connecting to www.itspaces.tech (www.itspaces.tech)|51.38.177.87|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88553500 (84M) [application/zip]
Saving to: ‘apex.zip’

apex.zip            100%[===================>]  84.45M  11.7MB/s    in 7.3s    

2023-10-22 15:23:01 (11.6 MB/s) - ‘apex.zip’ saved [88553500/88553500]

Archive:  apex.zip
   creating: apex/
   creating: apex/examples/
   creating: apex/examples/imagenet/
  inflating: apex/examples/imagenet/main_amp.py  
  inflating: apex/examples/imagenet/README.md  
   creating: apex/examples/docker/
  inflating: apex/examples/docker/README.md  
  inflating: apex/examples/docker/Dockerfile  
   creating: apex/examples/dcgan/
  inflating: apex/examples/dcgan/main_amp.py  
  inflating: apex/examples/dcgan/README.md  
  inflating: apex/examples/README.md  
   creating: apex/examples/simple/
 

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!git checkout 23.05
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
!pip install openpyxl
%cd ..
!pip install sentencepiece

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
HEAD is now at 0da3ffb use `nvfuser_codegen`
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
DEPRECATION: --build-option and --global-option are deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use --config-settings. Discussion can be found at https://github.com/pypa/pip/issues/11859
Processing /content/apex
  Running command python setup.py egg_info


  torch.__version__  = 2.1.0+cu118


  running egg_info
  creating /tmp/pip-pip-egg-info-6582uvmn/apex.egg-info
  writing /tmp/pip-pip-egg-info-6582uvmn/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-6582uvmn/apex.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-6582uvmn/apex.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-6582uvmn/apex.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg

In [ ]:
import torch
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging


logger = logging.getLogger()
device_cuda = torch.device("cuda")
# device_mps = torch.device("cpu")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
DATA_PATH = Path.cwd()/'data'
LOG_PATH = Path.cwd()/'logs'
MODEL_PATH = Path.cwd()/'model'
LABEL_PATH = Path.cwd()/'labels'

In [ ]:
DATA_PATH.mkdir(parents=True, exist_ok=True)
LOG_PATH.mkdir(parents=True, exist_ok=True)
MODEL_PATH.mkdir(parents=True, exist_ok=True)
LABEL_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
# Lecture de la base de donnees et selection des colonnes
data = pd.read_excel("data2.xlsx")
data = data[[c for c in data.columns if c!='Unnamed: 0']]
cols = data.columns.to_list()[7:]
cols.insert(0,'Commentaire')
data = data[cols]

In [ ]:
data.head(3)

,Commentaire,Cohésion Sociale,Insecurité,Gouvernance économique,Gouvernance politique
0,"D’accord avec l’internaute Nr 1, le Developpem...",1,0,1,0
1,"Il m’apparait singulier de comparer une ""coopé...",0,0,0,1
2,Encore une OCS. Je pense qu’on devrait serrer ...,1,0,0,0


In [ ]:
val_set = data.sample(frac=0.2, replace=False, random_state=42)
train_set = data.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv(DATA_PATH/'val.csv')
train_set.to_csv(DATA_PATH/'train.csv')

Nombre de commentaires dans le val_set: 76
Nombre de commentaires dans le train_set: 303


In [ ]:
# Creation du fichier labels.txt
labels = data.columns[1:].to_list()
with open(LABEL_PATH/'labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [ ]:
# Creation du fichier labels.csv
pd.DataFrame(labels).to_csv(LABEL_PATH/'labels.csv',index=False, header=False)

In [ ]:
all_texts = data['Commentaire'].to_list()
print('Nombre de commentaires:', len(all_texts))

Nombre de commentaires: 379


### Création de LMDataBunch

In [ ]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=16,
                    max_seq_length=512,
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [ ]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            fp16_opt_level="O2")

In [ ]:
lm_learner.fit(epochs=30,
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(210, 3.570679288818723)

In [ ]:
lm_learner.validate()

{'loss': 0.1361434906721115, 'perplexity': 1.1458462476730347}

In [ ]:
lm_learner.save_model()

# Entrainement du model

### Création de databunch pour la classification

In [ ]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train.csv',
                          val_file='val.csv',
                          label_file='labels.txt',
                          text_col='Commentaire',
                          label_col=['Cohésion Sociale', 'Insecurité', 'Gouvernance économique', 'Gouvernance politique'],
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

In [ ]:
len(databunch.train_dl)

19

### Création de Learner

In [ ]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path.cwd()/'finetuned_model'

WGTS_PATH = MODEL_PATH/'model_out/pytorch_model.bin'

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
OUTPUT_DIR

PosixPath('/content/finetuned_model')

In [ ]:
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path=MODEL_PATH/'model_out',
                        metrics=metrics,
                        device=device_cuda,
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=False,
                        logging_steps=50)

Some weights of the model checkpoint at /content/model/model_out were not used when initializing CamembertForMultiLabelSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForMultiLabelSequenceClassification were not initialized from the model checkpoint at /content/model/model_out and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bia

In [ ]:
cl_learner.fit(epochs=30,
            lr=9e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:261: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(570, 0.2459853463953263)

In [ ]:
cl_learner.validate()

{'loss': 0.8584385911623637,
 'return_preds': False,
 'fbeta': 0.7152089476585388,
 'roc_auc': 0.8241206030150754}

### Prédictions

In [ ]:
cl_learner.save_model()

In [ ]:
comments = [
    '''Le Burkina compte plus d’une soixantaine d’ethnies c’est pas seulement avec les mossis que les peuls rencontrent des soucis. Oui c’est vrai que les peuls des milieux ruraux ce sont trop mis a l’écart des autres communautés et font difficilement confiance au autres toujours a s’autostigmatiser''',
    '''Ah ! Ceci ressemble encore à des complaintes qui ne servent à rien . Les affaires sont les affaires. S’ils ont réfusé de vendre leurs armes , c’est que c’etait pas de bonnes affaires pour eux. C’est tout. Maintenant si vous voulez qu’on vende des armes à vos conditions alors que vous n’en produisez pas et que ces partenaires ne sont pas en obligation de se plier à ces conditions, il n’y’a pas à se plaindre. Que dire alors si des partenaires reclamaient que leurs armes ne soient pas payées en dévises courantes mais plutôt en lingots ou mines entiers ? Des situations comme Karma, Yirgou etc ne favorisent pas non plus la vente d’arme à nos pays car certains ne voudraient pas se retrouver embarqués . C’est ce que certains font semblant de ne pas savoir. Or nous savons qu’il faut l’autorisation des instances politiques pour des ventes d’armes dans certains pays
  Aucun de ces "partenaires" n’a pour priorité la satisfaction des réseaux sociaux et de la rue. Donc au lieu de continuer à se plaindre à chaque occasion, chose qui perdra à force de répétition son tranchant, mieux vaudra avancer dans le réalisme et loin de l’influence des tapages de réseaux sociaux.
''',
    '''SOYEZ DIGNE ET AYEZ DE L’HONNEUR MR KYELEM APRES TOUT CE QUE VOUS AVEZ FAIT COMME TAPAGE MEDIATIQUE POUR ETRE NOMME.
La composition du gouvernement vient de sortir. Le vin est tiré et il faut le boire mais à quel prix surtout pour le PM Mr Kyelem.
Nous avons deux observations à faire en ce qui concerne les principes et idéaux  prônés par Mr Kyelem :
1. Voilà ce que disait Mr Kyelem lors d’un débat public sur Mr Bassolma qu’il a accepté dans son gouvernement aujourdhui : « « « « il y avait une différence de style et de fonds entre Bassolma Bazié et Tolé Sagnon. Tolé Sagnon me semblait beaucoup plus convaincu de ce qu’ íl faisait et disait que par rapport à Bssolma Bazié.
Bassolma Bazié est quelqu ‘un, n’est-ce pas, qui était excité etc et ces gens-là …. cad quand vous le voyez dans les interviews etc il semblait quelqu’un d’excité, pas stable etc et moi je me méfie tout de suite de ces gens-là car ça traduit une certaine immaturité psychologique et intellectuelle……..….Quand on est mure on est calme, posé etc on est convaincu de ce que l’on fait. Vous avez vu Tolé Sagnon ici. Tolé Sagnon par exemple exposait ses problèmes calmement mais fermement. C’est comme un certain syndicaliste qui était Nanéma qui dirigeait le SYNTSHA il était aussi calme mais ferme………………………..Ça ne m’a jamais surpris, je n’ai jamais fait confiance à des gens comme ça (Bassolma Bazié)  qui sont capable de tout du pire comme du meilleur, donc moi ça ne m’a pas surpris…. » » » ».
Après ces graves insultes publiques faites par  Mr Kyelem sur Bassolma Bazié, voilà que Mr Bassolma est dans le gouvernement du PM Kyelem. D’abord insulter publiquement quelqu’un est un manque de reste pour la personne insultée et aussi à soi-même.
Comment va-t-il le regarder en face maintenant, ce Bassolma Bazié qu’ íl a qualifié d’excité, de pas stable, immature psychologiquement et intellectuellement, ….je me méfie de lui (Bassolma). Comment Mer Kyelem travaillera-t-il avec Mr Bassolma dans ces conditions ?
Quand on a un minimum d’honneur, de dignité, quand on a foi en ses idéaux et quand on est conséquent envers soi-même, alors Mr Kyelem devrait démissionner de son poste de PM et rendre le tablier.
2. Reconduction de Mme Olivia Rouamba
Mme Roumba est l’antipode des idées sankaristes déjà de par ces frasques à New York l’été dernier. Donc Olivia est contraire même aux idéaux prônes par Mr Kyelem.
Au regard de ses deux éléments si Mr Kyelem avait un minimum d’honneur, de dignité, de foi, il devrait démissionner immédiatement et surtout que Mr Bassolma Bazié a été élevé au poste de Ministre d’Etat apparait comme un désaveu aux idéaux de Mr Kyelem
Que pouvait-on attendre d’un débateur public comme Mr Kyelem ? ''',
]

In [ ]:
len(comments)

3

In [ ]:
for comment in comments:
  prediction = cl_learner.predict_batch(comment)[0]
  print(prediction)

[('Insecurité', 0.9435347318649292), ('Gouvernance politique', 0.1383087933063507), ('Gouvernance économique', 0.007052947301417589), ('Cohésion Sociale', 0.00447357352823019)]
[('Insecurité', 0.9301204085350037), ('Gouvernance politique', 0.1903814673423767), ('Gouvernance économique', 0.006377349141985178), ('Cohésion Sociale', 0.004145835991948843)]
[('Insecurité', 0.9376828074455261), ('Gouvernance politique', 0.18761494755744934), ('Gouvernance économique', 0.005737915635108948), ('Cohésion Sociale', 0.004278078675270081)]
